# Programmatic Tool Calling (Experimental)

Emulate Anthropic's Programmatic Tool Calling pattern in LangChain.

Ref:

- https://platform.claude.com/docs/agents-and-tools/tool-use/programmatic-tool-calling

**Problem**: Traditional tool calling requires API round-trips for each tool call.
With many tool calls, this adds latency and token overhead.

**Solution**: Let the agent generate Python code that calls multiple tools,
execute the code in a sandbox, and return only the final result.

**Benefits**:

- Reduced latency (batch multiple tool calls)
- Token efficiency (aggregate results before returning to model)
- Complex workflows (loops, conditionals over tool results)

Uses [sandbox-runtime](https://github.com/anthropic-experimental/sandbox-runtime) for secure execution.
No global install required - runs via `npx`.


## Setup


In [25]:
import json
import re
import shutil
import subprocess
import tempfile
from pathlib import Path
from typing import Any

import rich
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import BaseTool, tool
from langgraph.graph.state import CompiledStateGraph

load_dotenv()

# Check for srt - prefer global install, fallback to npx
SRT_CMD: list[str]
if shutil.which("srt"):
    SRT_CMD = ["srt"]
    rich.print("[green]srt found (global)[/green]")
elif shutil.which("npx"):
    SRT_CMD = ["npx", "@anthropic-ai/sandbox-runtime"]
    rich.print("[green]srt available via npx[/green]")
else:
    raise RuntimeError("srt not available. Install Node.js for npx support.")

srt available via npx

## Define Tool Registry

These tools can be called programmatically from generated code via `tool_call()`.


In [26]:
@tool
def query_sales(region: str) -> dict[str, Any]:
    """Query sales data for a region.

    Returns: {"region": str, "revenue": int, "units": int, "top_product": str}
    Error: {"error": str} if region not found.
    """
    data = {
        "west": {"revenue": 150000, "units": 1200, "top_product": "Widget A"},
        "east": {"revenue": 220000, "units": 1800, "top_product": "Widget B"},
        "central": {"revenue": 180000, "units": 1500, "top_product": "Widget A"},
        "north": {"revenue": 95000, "units": 800, "top_product": "Widget C"},
        "south": {"revenue": 130000, "units": 1100, "top_product": "Widget B"},
    }
    region_lower = region.lower()
    if region_lower in data:
        return {"region": region, **data[region_lower]}
    return {"error": f"Unknown region: {region}"}


@tool
def get_weather(city: str) -> dict[str, Any]:
    """Get current weather for a city.

    Returns: {"city": str, "temp": int, "condition": str, "humidity": int}
    Error: {"error": str} if city not found.
    """
    data = {
        "tokyo": {"temp": 22, "condition": "Sunny", "humidity": 45},
        "new york": {"temp": 18, "condition": "Cloudy", "humidity": 60},
        "london": {"temp": 15, "condition": "Rainy", "humidity": 80},
        "paris": {"temp": 20, "condition": "Partly Cloudy", "humidity": 55},
        "sydney": {"temp": 25, "condition": "Sunny", "humidity": 40},
    }
    city_lower = city.lower()
    if city_lower in data:
        return {"city": city, **data[city_lower]}
    return {"error": f"Unknown city: {city}"}


TOOL_REGISTRY: dict[str, BaseTool] = {t.name: t for t in [query_sales, get_weather]}

rich.print("Available tools:", list(TOOL_REGISTRY.keys()))

Available tools:
['query_sales', 'get_weather']

## Tool Search

Allows agent to discover available tools by pattern.


In [27]:
@tool
def tool_search(pattern: str) -> list[dict[str, Any]]:
    """Search for available tools by regex pattern.

    Args:
        pattern: Regex pattern to match against tool names and descriptions.

    Returns: list of tool schemas with properties, required, description, etc.
    Empty list if no tools found.
    """
    rich.print(f"[bold cyan]tool_search({pattern!r})[/bold cyan]")
    matches = []
    for name, t in TOOL_REGISTRY.items():
        if re.search(pattern, name, re.IGNORECASE) or re.search(pattern, t.description, re.IGNORECASE):
            if t.args_schema and hasattr(t.args_schema, "model_json_schema"):
                schema = t.args_schema.model_json_schema()
                schema["name"] = name  # Add tool name
                matches.append(schema)
    rich.print(f"[dim]{json.dumps(matches, indent=2)}[/dim]")
    return matches

## Sandboxed Code Execution Tool

Execute Python code in a sandbox with `tool_call()` to invoke registered tools.
Uses JSON-RPC over stdin/stdout to communicate with the host.


In [28]:
import queue
import threading


def create_sandboxed_execute_code_tool(registry: dict[str, BaseTool]) -> BaseTool:
    """Create a sandboxed code execution tool using srt."""

    @tool
    def execute_code(code: str) -> str:
        """Execute async Python code in a sandbox.

        Available functions:
        - await tool_call(name, **kwargs): Call a registered tool, returns dict
        - print(): Output results (only printed text is returned)

        Example:
            sales = await tool_call("query_sales", region="west")
            print(f"Revenue: {sales['revenue']}")
        """
        rich.print("[bold cyan]Executing sandboxed code:[/bold cyan]")
        rich.print(code)
        rich.print("[bold cyan]---[/bold cyan]")

        # Indent user code for async main()
        indented_code = "\n".join("    " + line for line in code.split("\n"))

        wrapper = f'''
import asyncio
import json
import sys

async def tool_call(name, **kwargs):
    """Call a tool on the host via stdout/stdin protocol."""
    request = {{"name": name, "kwargs": kwargs}}
    sys.stdout.write(f"__TOOL_REQUEST__{{json.dumps(request)}}__END_REQUEST__\\n")
    sys.stdout.flush()
    response_line = sys.stdin.readline().strip()
    return json.loads(response_line)

_output_lines = []
_original_print = print
def print(*args, **kwargs):
    import io
    buf = io.StringIO()
    kwargs["file"] = buf
    _original_print(*args, **kwargs)
    _output_lines.append(buf.getvalue())

async def main():
{indented_code}

asyncio.run(main())

sys.stdout.write("__USER_OUTPUT__\\n")
sys.stdout.flush()
for line in _output_lines:
    sys.stdout.write(line)
sys.stdout.flush()
'''

        with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False) as f:
            f.write(wrapper)
            script_path = f.name

        try:
            proc = subprocess.Popen(
                [*SRT_CMD, "python", "-u", script_path],
                stdin=subprocess.PIPE,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,
            )

            # Read stdout in separate thread to avoid blocking
            output_q: queue.Queue[str | None] = queue.Queue()

            def read_stdout() -> None:
                for line in proc.stdout:  # type: ignore[union-attr]
                    output_q.put(line)
                output_q.put(None)

            reader = threading.Thread(target=read_stdout)
            reader.start()

            output_lines: list[str] = []
            user_output_started = False

            while True:
                try:
                    line = output_q.get(timeout=30)
                except queue.Empty:
                    break

                if line is None:
                    break

                if "__TOOL_REQUEST__" in line:
                    start = line.index("__TOOL_REQUEST__") + len("__TOOL_REQUEST__")
                    end = line.index("__END_REQUEST__")
                    request = json.loads(line[start:end])

                    tool_name = request["name"]
                    tool_kwargs = request["kwargs"]

                    if tool_name in registry:
                        call_result = registry[tool_name].invoke(tool_kwargs)
                        rich.print(f"[dim]  await tool_call({tool_name!r}, {tool_kwargs}) -> {call_result}[/dim]")
                    else:
                        call_result = {"error": f"Unknown tool: {tool_name}"}

                    proc.stdin.write(json.dumps(call_result) + "\n")  # type: ignore[union-attr]
                    proc.stdin.flush()  # type: ignore[union-attr]
                elif "__USER_OUTPUT__" in line:
                    user_output_started = True
                elif user_output_started:
                    output_lines.append(line)

            reader.join(timeout=5)
            proc.wait(timeout=5)
            stderr = proc.stderr.read()  # type: ignore[union-attr]

            output = "".join(output_lines)
            if stderr:
                output += f"\nStderr: {stderr}"
            if proc.returncode != 0:
                output += f"\nExit code: {proc.returncode}"

            return output.strip() if output.strip() else "Code executed successfully (no output)"

        except subprocess.TimeoutExpired:
            proc.kill()
            return "Error: Execution timed out (30s limit)"
        except Exception as e:
            return f"Error: {type(e).__name__}: {e}"
        finally:
            Path(script_path).unlink(missing_ok=True)

    return execute_code


execute_code_tool = create_sandboxed_execute_code_tool(TOOL_REGISTRY)
rich.print("Sandboxed execute_code tool created")

Sandboxed execute_code tool created

## Create Agent


In [29]:
model = ChatAnthropic(model="claude-sonnet-4-5-20250929")

SYSTEM_PROMPT = """You are a data analyst assistant.

You have access to:
- tool_search: Search for available tools by regex pattern
- execute_code: Run Python code in a secure sandbox with tool_call(name, **kwargs)

Workflow:
1. Use tool_search to find relevant tools
2. Use execute_code with tool_call() to invoke them in batch
3. Aggregate results and print the final summary"""

agent: CompiledStateGraph[Any] = create_agent(
    model=model,
    tools=[tool_search, execute_code_tool],
    system_prompt=SYSTEM_PROMPT,
)

rich.print("Agent created")

Agent created

## Test: Multi-Region Analysis


In [30]:
query = """For each region, get sales data and check weather in its capital city.
Capital cities: West=Sydney, East=New York, Central=London, North=Tokyo, South=Paris.

Calculate revenue per unit for each region, then recommend the best region to visit
considering both business performance (high revenue per unit) and weather (sunny preferred)."""

result = agent.invoke({"messages": [{"role": "user", "content": query}]})

rich.print("\n[bold green]Final response:[/bold green]")
rich.print(result["messages"][-1].content)

tool_search('sales')

tool_search('weather')

[
  {
    "description": "Query sales data for a region.\n\nReturns: {\"region\": str, \"revenue\": int, \"units\": int, 
\"top_product\": str}\nError: {\"error\": str} if region not found.",
    "properties": {
      "region": {
        "title": "Region",
        "type": "string"
      }
    },
    "required": [
      "region"
    ],
    "title": "query_sales",
    "type": "object",
    "name": "query_sales"
  }
]

[
  {
    "description": "Get current weather for a city.\n\nReturns: {\"city\": str, \"temp\": int, \"condition\": str, 
\"humidity\": int}\nError: {\"error\": str} if city not found.",
    "properties": {
      "city": {
        "title": "City",
        "type": "string"
      }
    },
    "required": [
      "city"
    ],
    "title": "get_weather",
    "type": "object",
    "name": "get_weather"
  }
]

Executing sandboxed code:

...

  await tool_call('get_weather', {'city': 'Tokyo'}) -> {'city': 'Tokyo', 'temp': 22, 'condition': 'Sunny', 
'humidity': 45}

  await tool_call('query_sales', {'region': 'South'}) -> {'region': 'South', 'revenue': 130000, 'units': 1100, 
'top_product': 'Widget B'}

  await tool_call('get_weather', {'city': 'Paris'}) -> {'city': 'Paris', 'temp': 20, 'condition': 'Partly Cloudy', 
'humidity': 55}

Final response:

## Summary

Based on my analysis of sales data and weather conditions across all five regions, **I recommend visiting the West 
region (Sydney)**.

### Key Findings:

**West Region - Sydney** is the clear winner with a perfect score of 10/10 because it excels in both criteria:
- **Highest Revenue per Unit**: $125.00 (best business performance)
- **Sunny Weather**: Perfect conditions for your visit

**Runner-up**: North (Tokyo) scored 9.70/10 with good business performance ($118.75 per unit) and sunny weather, 
but slightly lower revenue per unit.

**Other Regions**:
- **East (New York)**: Strong revenue ($122.22/unit) but cloudy weather
- **Central (London)**: Decent revenue ($120.00/unit) but rainy weather
- **South (Paris)**: Lower revenue ($118.18/unit) with partly cloudy conditions

The West region offers the optimal combination of the highest business performance and ideal weather conditions, 
making it the best choice for your visit!